<a href="https://colab.research.google.com/github/Natan1995/TFM_VIU/blob/main/Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experimentos - Ánalisis de sentiemiento con LLM's<br>
                                        

### Instalación de librerias

In [ ]:
%%capture
!pip install -q transformers sentencepiece

In [ ]:
%%capture
!pip install accelerate -U

In [ ]:
%%capture
!pip install datasets

In [ ]:
%%capture
!pip install --upgrade transformers
!pip install --upgrade sentencepiece
#!pip install --upgrade TensorFlow

In [ ]:
%%capture
!pip install nlpaug  python-dotenv
!pip install -U tensorflow==2.15

In [ ]:
!pip show TensorFlow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, tf_keras


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sympy import symbols, diff

import random
import itertools
import math
import warnings
import IPython

from google.colab import drive
from google.colab import files
import os
import zipfile

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
import nltk
import string
import nlpaug.augmenter.word as naw

In [ ]:
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset

In [ ]:
#from transformers import RobertaTokenizer, RobertaModel
#from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BigBirdForMaskedLM,BigBirdTokenizer,pipeline, Trainer, TrainingArguments
import sentencepiece

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",200)
pd.set_option("expand_frame_repr", True)
pd.set_option("max_info_columns", 200)
pd.set_option("display.float_format", '{:.2f}'.format )
#pd.set_option("precision", 2)

In [ ]:
plt.rcParams["figure.figsize"] = (8,8)

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98% }</style>"))

In [ ]:
# Cargar el archivo Kaggle JSON
uploaded = files.upload()

Saving data_calls.zip to data_calls.zip


In [ ]:
with zipfile.ZipFile('data_calls.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped_data')

# 1. Metodos de Apoyo

In [ ]:
def extract_zipFile(zip_file, destination_folder):
  # Crear la carpeta si no existe
  if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

  # Descomprimir el archivo
  with zipfile.ZipFile(zip_file, 'r') as zip_ref:
      zip_ref.extractall(destination_folder)
      print('Archivo descomprimido en la carpeta:', destination_folder)

In [ ]:
def load_archives(input_folder, type_files, format_files):

  data = []

  for filename in os.listdir(input_folder):
      if filename.endswith(type_files):
          file_path = os.path.join(input_folder, filename)

          with open(file_path, 'r', encoding = format_files) as file:
              content = file.read()

          nps = filename.split('_')[0]

          data.append([filename, nps, content])

  return pd.DataFrame(data, columns=['fileName', 'nps','content'])

In [ ]:
def stem_word(word):
    stemmer = SnowballStemmer("spanish")
    return stemmer.stem(word)

In [ ]:
def remove_punctuation(tokens):
    not_punctuation = [token for token in tokens if token not in string.punctuation]
    return not_punctuation

In [ ]:
def remove_stepwords(tokens):
    sw_spa = nltk.corpus.stopwords.words('spanish')

    not_stewords = [token for token in tokens if token not in sw_spa]
    return not_stewords

In [ ]:
def preprocess_text(conversation):

    punctuation = set(string.punctuation)
    stemmer = PorterStemmer()

    tokens = nltk.word_tokenize(conversation)
    tokens = [stemmer.stem(word) for word in tokens]

    tokens = remove_punctuation(tokens)
    tokens = remove_stepwords(tokens)

    return ' '.join(tokens)

In [ ]:
def preprocess_dataframe(df, text_column):

    df[text_column + '_processed'] = df[text_column].fillna('')
    df[text_column + '_processed'] = df[text_column].str.lower()
    df[text_column + '_processed'] = df[text_column].apply(preprocess_text)

    return df

In [ ]:
def get_llm_model(model_name):

  model_pipeline = pipeline(
    model = model_name,
    return_all_scores=True
  )

  return model_pipeline

In [ ]:
def get_nps(model,conversation):
  conversation = conversation[:512]
  obj = model(conversation)
  max_score_obj = max(obj[0], key=lambda x: x['score'])
  return max_score_obj['label']

In [ ]:
def get_aug_df(df_input, model_path, n_rows):

  # Inicializamos el modelo de aumento de texto con la ruta del modelo,
  # Se configura el modelo para sustituir palabras con las 500 principales predicciones, y una probabilidad de 0.7 de ser alterado.
  aug = naw.ContextualWordEmbsAug(model_path=model_path, action="substitute", top_k=500, aug_p=0.7)
  augmented_rows = []

  # Calcular el total de filas en el DataFrame de entrada y determinar cuántas veces se debe aumentar cada fila.
  total_rows = len(df_input)
  augmentations_needed = n_rows
  augmentations_per_row = max(1, augmentations_needed // total_rows) if total_rows else 0

  # Iteramos sobre cada fila del DataFrame de entrada.
  for _, row in df_input.iterrows():
    for i in range(0,augmentations_per_row):
      augmented_text = aug.augment(row['content'])

      # Añade la fila aumentada al listado
      augmented_rows.append({
                      'fileName': row['fileName'],  # Copia el mismo valor de filename
                      'nps': row['nps'],  # Copia el mismo valor de nps
                      'content': augmented_text  # Usa el contenido aumentado
                  })

  # Limitamos el número de filas aumentadas al número deseado y  creamos un DataFrame con estas.
  augmented_rows = augmented_rows[:n_rows]
  augmented_df = pd.DataFrame(augmented_rows)

 # Combinamos el DataFrame original con el nuevo DataFrame y reiniciamos el índice para mantener la consistencia
  consolidated_df = pd.concat([df_input, augmented_df]).reset_index(drop=True)
  consolidated_df['content'] = consolidated_df['content'].astype(str)

  return consolidated_df

In [ ]:
def train_model_llm(df_input, model_name):

  etiquetas_a_numeros = {'Positivo': 2, 'Neutro': 1, 'Negativo': 0}

  # Preprocesamiento
  df_input['content'] = df_input['content'].apply(lambda x: str(x))
  df_input['nps'] = df_input['nps'].map(etiquetas_a_numeros)
  df_input = df_input.loc[:,["content","nps"]]

  # Dividir el dataset
  train_df, test_df = train_test_split(df_input, test_size=0.2)

  # Definimos el modelo
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # Primero, tokeniza tus datos. Este es un paso simplificado. Puedes necesitar ajustar según tus datos.
  def tokenize_function(examples):
      return tokenizer(examples["content"], padding="max_length", truncation=True)

  # Crear objetos Dataset para entrenamiento y prueba
  train_dataset = Dataset.from_pandas(train_df)
  test_dataset = Dataset.from_pandas(test_df)

  train_dataset = train_dataset.map(tokenize_function, batched=True)
  test_dataset = test_dataset.map(tokenize_function, batched=True)

  # Renombrar la columna de etiquetas y eliminar columnas no necesarias
  train_dataset = train_dataset.rename_column("nps", "labels").remove_columns(["content", "__index_level_0__"])
  test_dataset = test_dataset.rename_column("nps", "labels").remove_columns(["content", "__index_level_0__"])

  # Definir argumentos de entrenamiento
  training_args = TrainingArguments(
      output_dir='./results',          # directorio para guardar los checkpoints del modelo
      num_train_epochs=10,              # número total de épocas de entrenamiento
      per_device_train_batch_size=8,   # tamaño del batch por dispositivo durante el entrenamiento
      per_device_eval_batch_size=8,    # tamaño del batch para la evaluación
      warmup_steps=500,                # número de pasos de warmup antes del entrenamiento
      weight_decay=0.01,               # fuerza del weight decay
      logging_dir='./logs',            # directorio para guardar los logs
      evaluation_strategy="epoch",
  )

  # Inicializar el Trainer
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset
  )

  trainer.train()

  # Guardar el modelo
  model_path = './model_save' # Aquí puedes especificar la ruta deseada
  trainer.save_model(model_path) # Guardar el modelo en la ruta especificada
  tokenizer.save_pretrained(model_path) # También es buena idea guardar el tokenizer junto con el modelo

In [ ]:
def predict_from_local_path(df_input, model_path):

  # Cargamos el modelo y el tokenizador
  model = AutoModelForSequenceClassification.from_pretrained(model_path)
  tokenizer = AutoTokenizer.from_pretrained(model_path)

  tokenized_data = tokenizer(df_input['content'].tolist(), padding=True, truncation=True, return_tensors="pt")

  # Modelo en modo evaluacion
  model.eval()

  # Realizar las predicciones
  with torch.no_grad(): # Esto asegura que el código se ejecuta sin calcular gradientes, lo cual es necesario para la inferencia
      outputs = model(**tokenized_data)
      preds = outputs.logits.argmax(-1).numpy() # Obtiene el índice de la mayor probabilidad

  # Si necesitas convertir preds a etiquetas textuales puedes hacerlo aquí
  etiquetas = {2: 'Positivo', 1: 'Neutro', 0: 'Negativo'}
  preds_etiquetas = [etiquetas[pred] for pred in preds]

  df_input['nps_adhoc'] = preds_etiquetas


  return df_input

# 2. Carga de Datos

In [ ]:
extract_zipFile('data_calls.zip', 'data_calls_folder')

Archivo descomprimido en la carpeta: data_calls_folder


In [ ]:
df_calls = load_archives('data_calls_folder', '.txt', 'ISO-8859-1')

In [ ]:
df_calls[:3]

,fileName,nps,content
0,Negativo_[965021512]_MERIDIANO_900_GEN._2023-03-04_15-34-14.txt,Negativo,"XXXXXX. XXXXXX. Hola, buenas tardes. Doña XXXXXX, soy XXXXXX de la Central de Asistencia de Seguros Meridiano. XXXXXX. XXXXXX, hola, buenas tardes. XXXXXX. XXXXXX. Hola, buenas tardes. Mire, de l..."
1,Positivo_[965021512]_MERIDIANO_900_GEN._2023-02-28_15-57-35.txt,Positivo,"de asistencia. En primer lugar, nuestras más sinceras condolencias por el fallecimiento de su padre, don XXXXXX. El motivo de mi llamada era comprobar si el servicio de su padre se había desarrol..."
2,Positivo_[965641606]_MERIDIANO_902_2023-02-28_10-32-31.txt,Positivo,"Dígame. Buenos días. Hablo con doña XXXXXX. Sí, soy yo. Mi nombre es XXXXXX Ley y llamo del Departamento de Asistencia de Seguros Meridiano. Sí, dígame. En primer lugar, permítame trasladarle mis..."


In [ ]:
df_calls.shape

(20, 3)

# 3. Preprocesamiento

In [ ]:
df_calls_f = get_aug_df(df_calls, 'bert-base-uncased', 500)
df_calls_f.shape

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

(520, 3)

In [ ]:
df_calls_f = preprocess_dataframe(df_calls_f,'content')

In [ ]:
df_calls_f[:3]

,fileName,nps,content,content_processed
0,Negativo_[965021512]_MERIDIANO_900_GEN._2023-03-04_15-34-14.txt,Negativo,"XXXXXX. XXXXXX. Hola, buenas tardes. Doña XXXXXX, soy XXXXXX de la Central de Asistencia de Seguros Meridiano. XXXXXX. XXXXXX, hola, buenas tardes. XXXXXX. XXXXXX. Hola, buenas tardes. Mire, de l...",xxxxxx xxxxxx hola buena tard doña xxxxxx xxxxxx central asistencia seguro meridiano xxxxxx xxxxxx hola buena tard xxxxxx xxxxxx hola buena tard mire hemo hablado ust mañana procedido informar coo...
1,Positivo_[965021512]_MERIDIANO_900_GEN._2023-02-28_15-57-35.txt,Positivo,"de asistencia. En primer lugar, nuestras más sinceras condolencias por el fallecimiento de su padre, don XXXXXX. El motivo de mi llamada era comprobar si el servicio de su padre se había desarrol...",asistencia primer lugar má sincera condolencia fallecimiento padr don xxxxxx motivo llamada comprobar si servicio padr desarrollado correctament si pudiéramo hacer asistirl ¿tenían alguna duda pro...
2,Positivo_[965641606]_MERIDIANO_902_2023-02-28_10-32-31.txt,Positivo,"Dígame. Buenos días. Hablo con doña XXXXXX. Sí, soy yo. Mi nombre es XXXXXX Ley y llamo del Departamento de Asistencia de Seguros Meridiano. Sí, dígame. En primer lugar, permítame trasladarle mis...",dígame bueno día hablo doña xxxxxx nombr xxxxxx ley llamo departamento asistencia seguro meridiano dígame primer lugar permítam trasladarl má sincera condolencia pérdida cámara mucha gracia motivo...


In [ ]:
df_calls_f.shape

(520, 4)

# 4. Implementación de modelos

### 4.2. Distilbert Base Model

Url HugginFace:  https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student

In [ ]:
df_calls.iloc[2]['content']

' Dígame. Buenos días. Hablo con doña XXXXXX. Sí, soy yo. Mi nombre es XXXXXX Ley y llamo del Departamento de Asistencia de Seguros Meridiano. Sí, dígame. En primer lugar, permítame trasladarle mis más sinceras condolencias por la pérdida de la cámara. Muchas gracias. Mi motivo de mi llamada era para saber si todo el servicio se estaba desarrollando correctamente y conforma los deseos de la familia. Sí, sí, todo correctamente. Todo perfecto. Nosotros estamos llegando al pueblo y allí ya está mi hermano y dice que todo perfecto, me ha comentado. Así que no hay ninguna incidencia ni nada de eso para mencionar. De acuerdo. Doña XXXXXX, también quería indicarle que vamos a enviar la gestoría para que la llame en unos 10 a 15 días el abogado para ayudarles a arreglar todos los trámites que tengan ahora con la Administración Pública. Vale, de acuerdo. Entonces lo confirmamos y le llamará eso. En un periodo de unos 10 a 15 días va a recibir usted un mensaje con los trámites que se cubren. Tie

In [ ]:
# Cargamos el modelo LLM a utilizar
model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
distilled_student_sentiment_classifier = get_llm_model(model_name)

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
df_calls['nps_dbml_1'] = df_calls['content'].apply(lambda x: get_nps(model=distilled_student_sentiment_classifier, conversation=x))
#df_calls['nps_dbml_2'] = df_calls['content_processed'].apply(lambda x: get_nps(model=distilled_student_sentiment_classifier, conversation=x))

In [ ]:
df_calls.loc[:, ['fileName','nps','nps_dbml_1']]

,fileName,nps,nps_dbml_1
0,Negativo_[965021512]_MERIDIANO_900_GEN._2023-03-04_15-34-14.txt,Negativo,positive
1,Positivo_[965021512]_MERIDIANO_900_GEN._2023-02-28_15-57-35.txt,Positivo,positive
2,Positivo_[965641606]_MERIDIANO_902_2023-02-28_10-32-31.txt,Positivo,positive
3,Neutro_[965021512]_MERIDIANO_900_GEN._2023-02-27_10-32-05.txt,Neutro,positive
4,Positivo_[965021512]_MERIDIANO_900_GEN._2023-02-26_22-09-50.txt,Positivo,negative
5,Positivo_[965641606]_MERIDIANO_902_2023-02-28_10-43-08.txt,Positivo,positive
6,Neutro_[965021512]_MERIDIANO_900_GEN._2023-02-27_10-18-51.txt,Neutro,positive
7,Positivo_[965641606]_MERIDIANO_902_2023-02-28_09-12-59_617980.txt,Positivo,positive
8,Positivo_[965641606]_MERIDIANO_902_2023-02-27_13-15-09.txt,Positivo,positive
9,Positivo_[965641606]_MERIDIANO_902_2023-02-28_11-05-56.txt,Positivo,positive


In [ ]:
# Entrenamiento del modelo adhoc
train_model_llm(df_calls_f, "lxyuan/distilbert-base-multilingual-cased-sentiments-student")

Map:   0%|          | 0/416 [00:00<?, ? examples/s]

Map:   0%|          | 0/104 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.805032
2,No log,0.130568
3,No log,0.004482
4,No log,0.001589
5,No log,0.000803
6,No log,0.000461
7,No log,0.000282
8,No log,0.000179
9,No log,0.000118
10,0.177000,0.000083


In [ ]:
df_calls_l = predict_from_local_path(df_calls, './model_save')
df_calls_l.loc[:, ['fileName','nps','nps_adhoc']]

,fileName,nps,nps_adhoc
0,Negativo_[965021512]_MERIDIANO_900_GEN._2023-03-04_15-34-14.txt,Negativo,Negativo
1,Positivo_[965021512]_MERIDIANO_900_GEN._2023-02-28_15-57-35.txt,Positivo,Positivo
2,Positivo_[965641606]_MERIDIANO_902_2023-02-28_10-32-31.txt,Positivo,Positivo
3,Neutro_[965021512]_MERIDIANO_900_GEN._2023-02-27_10-32-05.txt,Neutro,Neutro
4,Positivo_[965021512]_MERIDIANO_900_GEN._2023-02-26_22-09-50.txt,Positivo,Positivo
5,Positivo_[965641606]_MERIDIANO_902_2023-02-28_10-43-08.txt,Positivo,Positivo
6,Neutro_[965021512]_MERIDIANO_900_GEN._2023-02-27_10-18-51.txt,Neutro,Neutro
7,Positivo_[965641606]_MERIDIANO_902_2023-02-28_09-12-59_617980.txt,Positivo,Positivo
8,Positivo_[965641606]_MERIDIANO_902_2023-02-27_13-15-09.txt,Positivo,Positivo
9,Positivo_[965641606]_MERIDIANO_902_2023-02-28_11-05-56.txt,Positivo,Positivo
